# IEEE Fraud Feature Engineering and EDA
_By Nick Brooks_

- V1 - 20/08/2019 - First Commit <br>

**Aim:** <br>
Build Features for Credit Card Fraud Project

In [ ]:
# https://www.kaggle.com/arnocandel/python-datatable
# more information: http://github.com/h2oai/datatable
!pip install --upgrade https://s3.amazonaws.com/artifacts.h2o.ai/releases/ai/h2o/pydatatable/0.8.0.dev115/x86_64-centos7/datatable-0.8.0.dev115-cp36-cp36m-linux_x86_64.whl
    
# Latest Pandas version
!pip install -q 'pandas==0.25' --force-reinstall

In [ ]:
import datatable as dt
import pandas as pd
print("DataTable version:", dt.__version__)
print("Pandas version:", pd.__version__)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import os
from contextlib import contextmanager
import gc; gc.enable()
import pprint
import time

import datetime
import csv
import random

from sklearn import preprocessing

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print("Define DF Schema..")

target_var = 'isFraud'

emails = {'gmail': 'google',
'att.net': 'att',
'twc.com': 'spectrum',
'scranton.edu': 'other',
'optonline.net': 'other',
'hotmail.co.uk': 'microsoft',
'comcast.net': 'other',
'yahoo.com.mx': 'yahoo',
'yahoo.fr': 'yahoo',
'yahoo.es': 'yahoo',
'charter.net': 'spectrum',
'live.com': 'microsoft',
'aim.com': 'aol',
'hotmail.de': 'microsoft',
'centurylink.net': 'centurylink',
'gmail.com': 'google',
'me.com': 'apple',
'earthlink.net': 'other',
'gmx.de': 'other',
'web.de': 'other',
'cfl.rr.com': 'other',
'hotmail.com': 'microsoft',
'protonmail.com': 'other',
'hotmail.fr': 'microsoft',
'windstream.net': 'other',
'outlook.es': 'microsoft',
'yahoo.co.jp': 'yahoo',
'yahoo.de': 'yahoo',
'servicios-ta.com': 'other',
'netzero.net': 'other',
'suddenlink.net': 'other',
'roadrunner.com': 'other',
'sc.rr.com': 'other',
'live.fr': 'microsoft',
'verizon.net': 'yahoo',
'msn.com': 'microsoft',
'q.com': 'centurylink',
'prodigy.net.mx': 'att',
'frontier.com': 'yahoo',
'anonymous.com': 'other',
'rocketmail.com': 'yahoo',
'sbcglobal.net': 'att',
'frontiernet.net': 'yahoo',
'ymail.com': 'yahoo',
'outlook.com': 'microsoft',
'mail.com': 'other',
'bellsouth.net': 'other',
'embarqmail.com': 'centurylink',
'cableone.net': 'other',
'hotmail.es': 'microsoft',
'mac.com': 'apple',
'yahoo.co.uk': 'yahoo',
'netzero.com': 'other',
'yahoo.com': 'yahoo',
'live.com.mx': 'microsoft',
'ptd.net': 'other',
'cox.net': 'other',
'aol.com': 'aol',
'juno.com': 'other',
'icloud.com': 'apple'}


us_emails = ['gmail', 'net', 'edu']

In [ ]:
def reduce_mem_usage(df, verbose=True):
    print("\nMemeory Usage Before:")
    print(df.info(memory_usage = 'deep'))
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    
    print("Memeory Usage After:")
    print(df.info(memory_usage = 'deep'))
    return df

@contextmanager
def timer(name):
    """
    Time Each Process
    """
    t0 = time.time()
    yield
    print('\n[{}] done in {} Minutes'.format(name, round((time.time() - t0)/60,2)))


In [ ]:
def fraud_aggregate_function(dataframe):
    # Credit https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm-w-gpu
    dataframe = (dataframe.assign(    
        TransactionAmt_to_mean_card1 = (dataframe['TransactionAmt'] / dataframe.groupby(['card1'])['TransactionAmt'].transform('mean')),
        TransactionAmt_to_std_card1 = (dataframe['TransactionAmt'] / dataframe.groupby(['card1'])['TransactionAmt'].transform('std')),
        TransactionAmt_to_std_card4 = (dataframe['TransactionAmt'] / dataframe.groupby(['card4'])['TransactionAmt'].transform('std')),
        TransactionAmt_to_mean_card4 = (dataframe['TransactionAmt'] / dataframe.groupby(['card4'])['TransactionAmt'].transform('mean')),

        id_02_to_mean_card1 = (dataframe['id_02'] / dataframe.groupby(['card1'])['id_02'].transform('mean')),
        id_02_to_std_card1 = (dataframe['id_02'] / dataframe.groupby(['card1'])['id_02'].transform('std')),
        id_02_to_std_card4 = (dataframe['id_02'] / dataframe.groupby(['card4'])['id_02'].transform('std')),
        id_02_to_mean_card4 = (dataframe['id_02'] / dataframe.groupby(['card4'])['id_02'].transform('mean')),

        D15_to_mean_card1 = (dataframe['D15'] / dataframe.groupby(['card1'])['D15'].transform('mean')),
        D15_to_std_card1 = (dataframe['D15'] / dataframe.groupby(['card1'])['D15'].transform('std')),
        D15_to_mean_card4 = (dataframe['D15'] / dataframe.groupby(['card4'])['D15'].transform('mean')),
        D15_to_std_card4 = (dataframe['D15'] / dataframe.groupby(['card4'])['D15'].transform('std')),
        D15_to_mean_addr1 = (dataframe['D15'] / dataframe.groupby(['addr1'])['D15'].transform('mean')),
        D15_to_std_addr1 = (dataframe['D15'] / dataframe.groupby(['addr1'])['D15'].transform('std')),

        TransactionAmt_Log = np.log(dataframe['TransactionAmt'])
    ))
    # https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499
    for c in ['P_emaildomain', 'R_emaildomain']:
        dataframe[c + '_bin'] = dataframe[c].map(emails)
        dataframe[c + '_suffix'] = dataframe[c].map(lambda x: str(x).split('.')[-1])
        dataframe[c + '_suffix'] = dataframe[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    
    return dataframe
    
# Device Features
def id_split(dataframe):
    # https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm-w-gpu
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]

    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
    dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]

    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe
    
def fraud_preprocessing(debug = None):
    print("Starting Pre-Processing..")
    with timer("Load Tables"):
        folder_path = '../input/ieee-fraud-detection/'
        # parse all files
        with timer("Read Tables"):
            # Read with DataTables (H20.ML)
            train_identity = dt.fread(f'{folder_path}train_identity.csv')
            test_identity = dt.fread(f'{folder_path}test_identity.csv')
            train_transaction = dt.fread(f'{folder_path}train_transaction.csv')
            test_transaction = dt.fread(f'{folder_path}test_transaction.csv')

        # join frames
        with timer("Join Identity Tables"):
            train_identity.key = 'TransactionID'
            test_identity.key = 'TransactionID'
            train = train_transaction[:, :, dt.join(train_identity)]
            test = test_transaction[:, :, dt.join(test_identity)]

        with timer("To Pandas"):
            if debug:
                train = reduce_mem_usage(train.to_pandas().head(debug))
                test = reduce_mem_usage(test.to_pandas().head(debug))
            else:
                train = reduce_mem_usage(train.to_pandas())
                test = reduce_mem_usage(test.to_pandas())
        
        traindex = train.index
        testdex = test.index
        original_cols = train.columns
        
        # Get column groups
        prefix_cols = {}
        prefix = ['C','D','Device','M','Transaction','V','addr','card','dist','id']
        for i,p in enumerate(prefix):
            prefix_cols[p] = [x for x in train.columns.tolist() if x.startswith(prefix[i])]
        
    with timer("Train/Test Split Feature Engineering"):        
        # Encoding - count encoding for both train and test
        for feature in ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'id_36']:
            train[feature + '_count_full'] = train[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))
            test[feature + '_count_full'] = test[feature].map(pd.concat([train[feature], test[feature]], ignore_index=True).value_counts(dropna=False))

        # Encoding - count encoding separately for train and test
        for feature in ['id_01', 'id_31', 'id_33', 'id_36']:
            train[feature + '_count_dist'] = train[feature].map(train[feature].value_counts(dropna=False))
            test[feature + '_count_dist'] = test[feature].map(test[feature].value_counts(dropna=False))
        
        # Aggregated Features
        train = fraud_aggregate_function(train)
        test = fraud_aggregate_function(test)
        
        # Combine
        y = train[target_var].copy()
        df = pd.concat([train,test],axis = 0).reset_index()
        del train, test
        
    with timer("Whole Feature Engineering"):
        START_DATE = '2017-12-01'
        startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')    
        df = df.assign(
                # New feature - decimal part of the transaction amount
                TransactionAmt_decimal = ((df['TransactionAmt'] - df['TransactionAmt'].astype(int)) * 1000).astype(int),

                # Count encoding for card1 feature. 
                # Explained in this kernel: https://www.kaggle.com/nroman/eda-for-cis-fraud-detection
                card1_count_full = df['card1'].map(df['card1'].value_counts(dropna=False)),

                # https://www.kaggle.com/fchmiel/day-and-time-powerful-predictive-feature
                Transaction_day_of_week = np.floor((df['TransactionDT'] / (3600 * 24) - 1) % 7),
                Transaction_hour = np.floor(df['TransactionDT'] / 3600) % 24,

                TransactionDT = df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x))),
            )
        df = df.assign(
                # Time of Day
                dow = df['TransactionDT'].dt.dayofweek,
                year = df['TransactionDT'].dt.year,
                month = df['TransactionDT'].dt.month,
                hour = df['TransactionDT'].dt.hour,
                day = df['TransactionDT'].dt.day,
        
                # All NaN
                all_group_nan_sum = df.isnull().sum(axis=1) / df.shape[1],
                all_group_0_count = (df == 0).astype(int).sum(axis=1) / (df.shape[1] - df.isnull().sum(axis=1))
        )
        
        # Create Features based on anonymised prefix groups
        for p in prefix_cols:
            column_set = prefix_cols[p]
            # Take NA count
            df[p + "group_nan_sum"] = df[column_set].isnull().sum(axis=1) / df[column_set].shape[1]

            # Take SUM/Mean if numeric
            numeric_cols = [x for x in column_set if df[x].dtype != object]
            if numeric_cols:
                df[p + "group_sum"] = df[column_set].sum(axis=1)
                df[p + "group_mean"] = df[column_set].mean(axis=1)
                # Zero Count
                df[p + "group_0_count"] = (df[column_set] == 0).astype(int).sum(axis=1) / (df[column_set].shape[1] - df[p + "group_nan_sum"])
                
    with timer("Rolling Features"):
        prefix = ['C', 'card']
        
        value = "TransactionAmt"
        timevar = "TransactionDT"
        
        for window in ['12h', '5d']:
            with timer("TimeFrame: {}".format(window)):
                for i, p in enumerate(prefix):
                    for var in prefix_cols[p]:
                        gb_var = [var]
                        df = pd.merge(df, (df.set_index(timevar)
                                           .sort_values(timevar)
                                           .groupby(gb_var)
                                           .rolling(window)[value].sum()
                                           .rename(gb_var[0] + "_AMT_" + window + "_sum")
                                           .reset_index()
                                           .drop_duplicates([timevar] + gb_var)),
                                      on= [timevar] + gb_var, how= 'left')

                        df = pd.merge(df, (df.set_index(timevar)
                                           .sort_values(timevar)
                                           .groupby(gb_var)
                                           .rolling(window)[value].mean()
                                           .rename(gb_var[0] + "_AMT_" + window + "_mean")
                                           .reset_index()
                                           .drop_duplicates([timevar] + gb_var)),
                                      on= [timevar] + gb_var, how= 'left')

                        df = pd.merge(df, (df.set_index(timevar)
                                           .sort_values(timevar)
                                           .groupby(gb_var)
                                           .rolling(window)[value].count()
                                           .rename(gb_var[0] + "_AMT_" + window + "_count")
                                           .reset_index()
                                           .drop_duplicates([timevar] + gb_var)),
                                      on= [timevar] + gb_var, how= 'left')

    with timer("Label Encode"):
        categorical_cols = []
        # Label Encoding
        for f in df.columns:
            if df[f].dtype=='object': 
                categorical_cols += [f]
                lbl = preprocessing.LabelEncoder()
                df[f] = lbl.fit_transform(df[f].astype(str))
                
                
    df.fillna(-9,inplace=True)
    df.set_index("TransactionID",inplace=True)
    
    # One more memory reduction
    df = reduce_mem_usage(df)
    print("Total Shape: {} Rows, {} Columns".format(*df.shape))
    
    return df, y, original_cols

In [ ]:
DEBUG = None # None for no debug, else number of rows
df, y, original_cols = fraud_preprocessing(debug = DEBUG)
df.loc[df.isFraud == 2, 'isFraud'] = np.nan

In [ ]:
with timer("Write Table"):
    write_cols = [x for x in df.columns if x not in original_cols] + [target_var]
    print("Writing {} Column".format(len(write_cols)))
    df.loc[df[target_var].notnull(),write_cols].to_csv("train_fraud_fe_nb.csv", index = True)
    df.loc[df[target_var].isnull(),write_cols].to_csv("test_fraud_fe_nb.csv", index = True)

In [ ]:
# Features for EDA
df['yrmth'] = df.year.astype(str) + df.month.map("{:02}".format)
df.loc[df.isFraud == 2, 'isFraud'] = np.nan
df['traintest'] = 'Test'
df.loc[df.isFraud.notnull(),'traintest'] = 'Train'

In [ ]:
print("Are there redundant Transaction IDs?")
print(df.index.value_counts().value_counts())

### Train / Submission Time Split

In [ ]:
f, ax = plt.subplots(2,2, figsize = [12,10])
for tt in ['Train','Test']:
    df.loc[df.traintest == tt,['all_group_nan_sum','TransactionDT']].set_index('TransactionDT')\
        .resample('1d').count().plot(label = tt, ax = ax[0,0])
    df.loc[df.traintest == tt,['all_group_nan_sum','TransactionDT']].set_index('TransactionDT')\
        .resample('1d').mean().plot(label = tt, ax = ax[1,0])
    df.loc[df.traintest == tt,['all_group_0_count','TransactionDT']].set_index('TransactionDT')\
        .resample('1d').mean().plot(label = tt, ax = ax[1,1])
    df.loc[df.traintest == tt,['TransactionAmt','TransactionDT']].set_index('TransactionDT')\
        .resample('1d').mean().plot(label = tt, ax = ax[0,1])
ax[0,0].set_title("Observation Count: Train/ Test")
ax[0,0].set_ylabel("Count")
ax[1,0].set_title("Average Number of Missing Values in Rows: Train/ Test")
ax[1,0].set_ylabel("Percent of Rows Is Null")
ax[0,1].set_title("Average Number of Missing Values in Rows: Train/ Test")
ax[0,1].set_ylabel("Percent of Rows Is Null")
ax[1,1].set_title("Average Number of Zero Values in Rows: Train/ Test")
ax[1,1].set_ylabel("Percent of Rows Is Zero")

plt.tight_layout(pad=0)
plt.show()

### Missing Values and Zeroes

In [ ]:
f, ax = plt.subplots(1,2,figsize = [12,5])

prefix = ['C','D','Device','M','Transaction','V','addr','card','dist','id']
for i, p in enumerate(prefix):
    df.loc[df.traintest == tt,[p + "group_nan_sum",'TransactionDT']].set_index('TransactionDT')\
        .resample('1d').mean().plot(label = "Missing", ax = ax[0])
    df.loc[df.traintest == tt,[p + "group_0_count",'TransactionDT']].set_index('TransactionDT')\
        .resample('1d').mean().plot(label = "Zero", ax = ax[1])

ax[0].get_legend().remove()
ax[1].legend(prefix,fontsize='large', loc='center left',bbox_to_anchor=(1, 0.5))

ax[0].set_title("Proportion of Data Missing by Column Group")
ax[1].set_title("Proportion of Data Equal Zero by Column Group")
ax[0].set_ylabel("Proportion Missing")
ax[1].set_ylabel("Proportion Zero")

plt.tight_layout(pad=1)
plt.show()

### Rolling Averages

### Univariate Exploration

In [ ]:
cols = ['card1_count_full', 'card1','card2','card2_count_full']
plot_df = df[cols + ['isFraud']]

t_r,t_c = 2, 2
f, axes = plt.subplots(t_r,t_c, figsize = [12,8],sharex=False, sharey=False)
row,col = 0,0
for c in cols:
    if col == t_c:
        col = 0
        row += 1
    sns.kdeplot(plot_df.loc[plot_df.isFraud == 0, c], shade = True, alpha = 0.6, color = 'black', ax = axes[row,col], label = 'Not Fraud')
    sns.kdeplot(plot_df.loc[plot_df.isFraud == 1, c], shade = True, alpha = 0.6, color = 'lime', ax = axes[row,col], label = 'Fraud')
    axes[row,col].set_title('{} and Fraud Distribution'.format(c.title()))
    col+=1
    
plt.tight_layout(pad=0)
plt.show()
del plot_df

In [ ]:
# Missing Values Pattern
# Hourly Pattern
# Individual's susepticality to fraud (explore ID)
# Is there a way to see how close various fraud claims are?

# Create a CPU kernel where I can experiment with features and LOFO..
# Smash all data together.